fyzCgTU9Iq9Fq1kj

In [49]:
!pip install -U -q google-generativeai langchain langchain-community langchain-google-genai
!pip install pymongo

In [59]:
import os
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [60]:
from google.colab import userdata

mongodb_connection_string = userdata.get("GoogleAPIKey")
gemini_api_key = userdata.get("GoogleAPIKey")


# Using HuggingFace Model

In [61]:
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5" , trust_remote_code = True)

## Setting up the database
To query a database, you first need to set one up.

1. **Load the California Housing Dataset:** Load the dataset from sklearn.datasets and extract it into a DataFrame.


In [62]:
from sklearn.datasets import fetch_california_housing

california_housing_bunch = fetch_california_housing(as_frame=True)
california_housing_df = california_housing_bunch.frame
records = california_housing_df.to_dict(orient="records")

2. **Connect to the mongo database and insert records:**

In [ ]:
client = MongoClient(mongodb_connection_string)
db = client["sample_db"]
collection = db["housing_data"]
collection.insert_many(records)
print("DataFrame successfully inserted into MongoDB.")

# Creating RAG Pipeline with Langchain

In [ ]:
llm = ChatGoogleGenerativeAI(model ="gemini-1.5-flash", google_api_key = gemini_api_key)

In [64]:
model = llm.GenerativeModel('gemini-pro')

def get_mongodb_query_from_llm(user_prompt, schema_description):
    prompt_template = f"""
    You are an AI assistant that converts natural language requests into MongoDB queries.
    Given the following MongoDB collection schema:
    {schema_description}

    Convert the following user request into a MongoDB find query (JSON format):
    User request: "{user_prompt}"
    MongoDB Query:
    """
    response = model.generate_content(prompt_template)
    return response.text.strip()

# Example usage
schema_desc = """Summary of housing data"""
user_request = "What is the median house price?"
generated_query_json = get_mongodb_query_from_llm(user_request, schema_desc)
print(f"Generated MongoDB Query: {generated_query_json}")

AttributeError: 'ChatGoogleGenerativeAI' object has no attribute 'GenerativeModel'